In [ ]:
# this cell just redirects GCR and GCRCatalogs to my own clones of master

import sys
import os
home_dir = '/global/u1/d/danielsf/'
gcr_dir = os.path.join(home_dir,'generic-catalog-reader-yymao')
assert os.path.isdir(gcr_dir)
gcr_cat_dir = os.path.join(home_dir, 'gcr-catalogs-desc')
assert os.path.isdir(gcr_cat_dir)
sys.path.insert(0,gcr_dir)
sys.path.insert(0,gcr_cat_dir)

In [ ]:
import healpy
import numpy as np
import GCRCatalogs
from lsst.sims.utils import angularSeparation
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
truth_catalog = GCRCatalogs.load_catalog('dc2_truth_run1.1')

In [ ]:
truth_catalog.list_all_quantities(include_native=True)

In [ ]:
dir(truth_catalog)

In [ ]:
for qty in ['ra', 'dec', 'redshift', 'healpix_2048',
            'agn', 'star', 'sprinkled',
            'u', 'g', 'r', 'i', 'z', 'y']:
    info_dict = truth_catalog.get_quantity_info(qty)
    print(qty,info_dict)

In [ ]:
center_ra = 54.6
center_dec = -28.0
radius = 0.2

center_ra_rad = np.radians(center_ra)
center_dec_rad = np.radians(center_dec)

center_vec = np.array([np.cos(center_dec_rad)*np.cos(center_ra_rad),
                       np.cos(center_dec_rad)*np.sin(center_ra_rad),
                       np.sin(center_dec_rad)])

list_of_healpix = healpy.query_disc(2048, center_vec, np.radians(radius), nest=True, inclusive=True)

def filter_on_healpix(hp):
    return np.array([hh in list_of_healpix for hh in hp])

def filter_on_dist(ra, dec):
    return angularSeparation(ra, dec, center_ra, center_dec)<radius

In [ ]:
bright_stars = truth_catalog.get_quantities(['ra', 'dec', 'mag_true_r'],
                                            native_filters=['star=1',
                                                            'r<17.0',
                                                            'healpix_2048<=%d' % list_of_healpix.max(),
                                                            'healpix_2048>=%d' % list_of_healpix.min()],
                                            filters=[(filter_on_healpix, 'healpix_2048'),
                                                     (filter_on_dist, 'ra', 'dec')])

In [ ]:
plt.scatter(bright_stars['ra'], bright_stars['dec'])

In [ ]:
bright_unsprinkled_galaxies = truth_catalog.get_quantities(['ra', 'dec', 'mag_true_r'],
                                                           native_filters=['star=0', 'sprinkled=0',
                                                                           'r<21.0',
                                                                           'healpix_2048<=%d' % list_of_healpix.max(),
                                                                           'healpix_2048>=%d' % list_of_healpix.min()],
                                                           filters=[(filter_on_healpix, 'healpix_2048'),
                                                                    (filter_on_dist, 'ra', 'dec')])

In [ ]:
plt.scatter(bright_unsprinkled_galaxies['ra'], bright_unsprinkled_galaxies['dec'])

In [ ]:
print(bright_unsprinkled_galaxies['mag_true_r'].min())